In [ ]:
import pandas as pd
df=pd.read_csv('/content/drive/MyDrive/data/train_cdna_data.csv')

<ipython-input-115-56eaeebe03a3>:2: DtypeWarning: Columns (7,12,13,15,16,17,18,19,29,37,45,46,47,56,58,66,69,71,74,228) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv('/content/drive/MyDrive/data/train_cdna_data.csv')


In [ ]:
import pandas as pd
from sklearn.feature_selection import VarianceThreshold
from scipy.stats import zscore
from tqdm import tqdm

# Load the CDNA dataset
cdna= df

# Display initial info
print("Initial Dataset Info:")
print(cdna.info())

# Step 1: Handle Missing Values
print("\nHandling Missing Values...")
# Remove columns with too many missing values (e.g., >50% missing)
missing_threshold = 0.5
missing_ratios = cdna.isnull().mean()
cdna = cdna.loc[:, missing_ratios < missing_threshold]

# Fill remaining NaNs (use median for numerical, mode for categorical)
for col in tqdm(cdna.columns, desc="Filling Missing Values"):
    if cdna[col].dtype in ['float64', 'int64']:
        cdna[col].fillna(cdna[col].median(), inplace=True)
    else:
        cdna[col].fillna(cdna[col].mode()[0], inplace=True)

# Step 2: Correct Data Types
print("\nCorrecting Data Types...")
for col in tqdm(cdna.columns, desc="Converting Data Types"):
    if cdna[col].dtype == 'object':
        try:
            cdna[col] = pd.to_numeric(cdna[col], errors='coerce')
            if cdna[col].isnull().sum() == 0:  # Converted to numeric successfully
                continue
        except:
            pass
        # Convert to category if not numeric
        cdna[col] = cdna[col].astype('category')

# Step 3: Remove Low-Variance Features
print("\nRemoving Low-Variance Features...")
numerical_cols = cdna.select_dtypes(include=['float64', 'int64']).columns
selector = VarianceThreshold(threshold=0.01)  # Threshold can be adjusted
cdna_num = selector.fit_transform(cdna[numerical_cols])
selected_columns = numerical_cols[selector.get_support()]
cdna = pd.concat([cdna[selected_columns], cdna.drop(columns=numerical_cols)], axis=1)

# # Step 4: Handle Outliers
# print("\nHandling Outliers...")
# z_threshold = 3
# for col in tqdm(cdna.select_dtypes(include=['float64', 'int64']), desc="Removing Outliers"):
#     cdna = cdna[(zscore(cdna[col]) < z_threshold) | (zscore(cdna[col]).isnull())]

# Step 5: Remove Highly Correlated Features
print("\nRemoving Highly Correlated Features...")
correlation_matrix = cdna.corr()
correlated_features = set()
threshold = 0.9  # Correlation threshold

for i in tqdm(range(len(correlation_matrix.columns)), desc="Checking Correlations"):
    for j in range(i):
        if abs(correlation_matrix.iloc[i, j]) > threshold:
            colname = correlation_matrix.columns[i]
            correlated_features.add(colname)

cdna = cdna.drop(columns=correlated_features)

# Final Dataset Info
print("\nProcessed Dataset Info:")
print(cdna.info())

# Save the processed CDNA dataset if needed
# cdna.to_csv('processed_cdna.csv', index=False)


Initial Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1285402 entries, 0 to 1285401
Columns: 303 entries, CUSTOMER_CODE to batch_date
dtypes: bool(9), float64(181), int64(35), object(78)
memory usage: 2.8+ GB
None

Handling Missing Values...


Filling Missing Values:   0%|          | 0/109 [00:00<?, ?it/s]<ipython-input-116-f6a51266135d>:25: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  cdna[col].fillna(cdna[col].mode()[0], inplace=True)
<ipython-input-116-f6a51266135d>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cdna[col].fillna(cdna[col].mode()[0], inplace=True)
Filling Miss


Correcting Data Types...


Converting Data Types:   0%|          | 0/109 [00:00<?, ?it/s]<ipython-input-116-f6a51266135d>:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cdna[col] = pd.to_numeric(cdna[col], errors='coerce')
<ipython-input-116-f6a51266135d>:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cdna[col] = cdna[col].astype('category')
Converting Data Types: 100%|██████████| 109/109 [00:37<00:00,  2.92it/s]



Removing Low-Variance Features...

Removing Highly Correlated Features...


Checking Correlations: 100%|██████████| 103/103 [00:00<00:00, 1183.83it/s]



Processed Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1285402 entries, 0 to 1285401
Data columns (total 80 columns):
 #   Column         Non-Null Count    Dtype   
---  ------         --------------    -----   
 0   v24            1285402 non-null  float64 
 1   v32            1285402 non-null  float64 
 2   v44            1285402 non-null  float64 
 3   v80            1285402 non-null  float64 
 4   v82            1285402 non-null  int64   
 5   v86            1285402 non-null  int64   
 6   v87            1285402 non-null  int64   
 7   v88            1285402 non-null  int64   
 8   v89            1285402 non-null  int64   
 9   v92            1285402 non-null  int64   
 10  v95            1285402 non-null  int64   
 11  v96            1285402 non-null  int64   
 12  v100           1285402 non-null  int64   
 13  v104           1285402 non-null  int64   
 14  v105           1285402 non-null  int64   
 15  v106           1285402 non-null  int64   
 16  v224       

In [ ]:
# Remove columns with more than 50% missing values
missing_threshold = 0.5
cdna = cdna.loc[:, cdna.isnull().mean() < missing_threshold]

# Explicitly remove columns with 100% missing values
cdna = cdna.loc[:, cdna.isnull().mean() < 1.0]

# Verify remaining columns
print(f"Remaining columns after removing high-NaN columns: {cdna.columns}")


Remaining columns after removing high-NaN columns: Index(['v24', 'v32', 'v44', 'v80', 'v82', 'v86', 'v87', 'v88', 'v89', 'v92',
       'v95', 'v96', 'v100', 'v104', 'v105', 'v106', 'v224', 'v225', 'v227',
       'v259', 'v263', 'v287', 'v288', 'v290', 'v291', 'v295', 'v298', 'v5',
       'v7', 'v30', 'v39', 'v40', 'v41', 'v45', 'v49', 'v50', 'v51', 'v52',
       'v53', 'v73', 'v229', 'v230'],
      dtype='object')


In [ ]:
cdna

,v24,v32,v44,v80,v82,v86,v87,v88,v89,v92,...,v41,v45,v49,v50,v51,v52,v53,v73,v229,v230
0,0.0,50.0,4.0,53.0,0,0,1,1,1,1,...,False,True,False,True,True,True,False,False,22101.0,22101.0
1,0.0,60.0,4.0,36.0,1,0,0,0,1,0,...,False,True,False,True,False,False,True,False,22101.0,23201.0
2,0.0,50.0,4.0,24.0,0,0,0,1,1,0,...,False,True,False,True,True,True,False,False,43552.0,43552.0
3,0.0,84.0,4.0,22.0,0,0,0,0,1,0,...,False,True,False,True,False,False,False,False,22102.0,22102.0
4,0.0,60.0,4.0,32.0,0,0,0,0,1,0,...,False,True,False,True,False,False,False,False,22101.0,22101.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1285397,1.0,50.0,4.0,34.0,0,0,0,0,1,0,...,False,True,False,True,False,False,False,False,21101.0,21101.0
1285398,0.0,60.0,4.0,46.0,1,0,1,1,1,1,...,False,True,False,True,False,False,False,False,22101.0,23201.0
1285399,0.0,29.0,4.0,32.0,1,0,0,0,1,0,...,False,True,False,True,True,True,True,False,22101.0,43552.0
1285400,0.0,50.0,4.0,33.0,0,0,0,0,1,0,...,False,True,False,True,True,True,False,False,22101.0,22101.0


In [ ]:
df1= pd.read_csv('/content/drive/MyDrive/data/Test/test_cdna_data.csv')

<ipython-input-119-fa339fb64d0d>:1: DtypeWarning: Columns (15,16,17,18,19,45,46,47,56,58,71,228) have mixed types. Specify dtype option on import or set low_memory=False.
  df1= pd.read_csv('/content/drive/MyDrive/data/Test/test_cdna_data.csv')


In [ ]:
df69= pd.read_csv('/content/drive/MyDrive/data/Test/test_cdna_data.csv')

<ipython-input-120-048b4def2766>:1: DtypeWarning: Columns (15,16,17,18,19,45,46,47,56,58,71,228) have mixed types. Specify dtype option on import or set low_memory=False.
  df69= pd.read_csv('/content/drive/MyDrive/data/Test/test_cdna_data.csv')


In [ ]:
# cdna['customer_code']=df1['CUSTOMER_CODE']
cdna_cleaned['batch_date']=df1['batch_date']

In [ ]:
cdna

,v24,v32,v44,v80,v82,v86,v87,v88,v89,v92,...,v41,v45,v49,v50,v51,v52,v53,v73,v229,v230
0,0.0,50.0,4.0,53.0,0,0,1,1,1,1,...,False,True,False,True,True,True,False,False,22101.0,22101.0
1,0.0,60.0,4.0,36.0,1,0,0,0,1,0,...,False,True,False,True,False,False,True,False,22101.0,23201.0
2,0.0,50.0,4.0,24.0,0,0,0,1,1,0,...,False,True,False,True,True,True,False,False,43552.0,43552.0
3,0.0,84.0,4.0,22.0,0,0,0,0,1,0,...,False,True,False,True,False,False,False,False,22102.0,22102.0
4,0.0,60.0,4.0,32.0,0,0,0,0,1,0,...,False,True,False,True,False,False,False,False,22101.0,22101.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1285397,1.0,50.0,4.0,34.0,0,0,0,0,1,0,...,False,True,False,True,False,False,False,False,21101.0,21101.0
1285398,0.0,60.0,4.0,46.0,1,0,1,1,1,1,...,False,True,False,True,False,False,False,False,22101.0,23201.0
1285399,0.0,29.0,4.0,32.0,1,0,0,0,1,0,...,False,True,False,True,True,True,True,False,22101.0,43552.0
1285400,0.0,50.0,4.0,33.0,0,0,0,0,1,0,...,False,True,False,True,True,True,False,False,22101.0,22101.0


In [ ]:
cdna['customer_code']=df['CUSTOMER_CODE']

In [ ]:
# Identify columns in CDNA with only one unique value
single_value_columns = [col for col in cdna.columns if cdna[col].nunique() == 1]

# Drop these columns from the dataset
cdna_cleaned = cdna.drop(columns=single_value_columns)

# Display the columns that were dropped
print(f"Columns dropped (unique value = 1): {single_value_columns}")

# Save the cleaned dataset
# cdna_cleaned.to_csv('cdna_cleaned.csv', index=False)

# Display information about the cleaned dataset
print(cdna_cleaned.info())


Columns dropped (unique value = 1): ['v39', 'v40', 'v41']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1285402 entries, 0 to 1285401
Data columns (total 40 columns):
 #   Column         Non-Null Count    Dtype   
---  ------         --------------    -----   
 0   v24            1285402 non-null  float64 
 1   v32            1285402 non-null  float64 
 2   v44            1285402 non-null  float64 
 3   v80            1285402 non-null  float64 
 4   v82            1285402 non-null  int64   
 5   v86            1285402 non-null  int64   
 6   v87            1285402 non-null  int64   
 7   v88            1285402 non-null  int64   
 8   v89            1285402 non-null  int64   
 9   v92            1285402 non-null  int64   
 10  v95            1285402 non-null  int64   
 11  v96            1285402 non-null  int64   
 12  v100           1285402 non-null  int64   
 13  v104           1285402 non-null  int64   
 14  v105           1285402 non-null  int64   
 15  v106           1285402 no

In [ ]:
cdna_cleaned['customer_code']=df['CUSTOMER_CODE']
cdna_cleaned2['batch_date']=df['batch_date']

In [ ]:
# import pandas as pd
# from sklearn.decomposition import PCA
# from sklearn.preprocessing import MinMaxScaler

# # Step 1: Load Dataset
# # cdna = pd.read_csv('/mnt/data/customer_cdna_first_100.csv')

# # Retain batch_date and customer_code for final output
# cdna_preprocessed = cdna[['batch_date', 'customer_code']].copy()

In [ ]:
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
from tqdm import tqdm

# Step 1: Load Dataset
# cdna = pd.read_csv('/content/drive/MyDrive/data/train_cdna_data.csv')

# Step 2: Identify Numerical Columns
numerical_cols = cdna_cleaned.select_dtypes(include=['float64', 'int64']).columns

# Step 3: Separate Binary and Non-Binary Numerical Columns
binary_cols = [col for col in numerical_cols if cdna[col].nunique() == 2]
non_binary_cols = [col for col in numerical_cols if col not in binary_cols]

print(f"Binary Columns: {binary_cols}")
print(f"Non-Binary Columns: {non_binary_cols}")

# Step 4: Scale Non-Binary Numerical Columns
scaler = MinMaxScaler()
cdna_cleaned[non_binary_cols] = scaler.fit_transform(cdna_cleaned[non_binary_cols])

# Step 5: Verify Scaling
print("Scaled Numerical Columns:")
print(cdna_cleaned[non_binary_cols].describe())

# Final Dataset Info
print("\nFinal Dataset Info:")
print(cdna_cleaned.info())

# Save Processed Dataset
# cdna.to_csv('processed_cdna_scaled.csv', index=False)


Binary Columns: ['v24', 'v82', 'v86', 'v87', 'v88', 'v89', 'v92', 'v95', 'v96', 'v104', 'v105', 'v106']
Non-Binary Columns: ['v32', 'v44', 'v80', 'v100', 'v224', 'v225', 'v227', 'v259', 'v263', 'v287', 'v288', 'v290', 'v291', 'v295', 'v298']
Scaled Numerical Columns:
                v32           v44           v80          v100          v224  \
count  1.285402e+06  1.285402e+06  1.285402e+06  1.285402e+06  1.285402e+06   
mean   5.281097e-02  9.008850e-01  2.931477e-01  1.987653e-02  5.498689e-02   
std    5.831806e-02  2.065985e-01  8.807706e-02  1.053138e-01  3.933557e-02   
min    0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00   
25%    3.374233e-02  1.000000e+00  2.320000e-01  0.000000e+00  3.267785e-02   
50%    5.010225e-02  1.000000e+00  2.720000e-01  0.000000e+00  4.822152e-02   
75%    5.419223e-02  1.000000e+00  3.360000e-01  0.000000e+00  6.528340e-02   
max    1.000000e+00  1.000000e+00  1.000000e+00  1.000000e+00  1.000000e+00   

               v225 

In [ ]:
len(cdna_cleaned['customer_code'].unique())

220699

In [ ]:
df

,CUSTOMER_CODE,v2,v3,v4,v5,v6,v7,v8,v9,v10,...,v294,v295,v296,v297,v298,v299,v300,v301,v302,batch_date
0,ab617a6a0a8582f4aaa1aeda38fd73377cb911e6096a98...,50-54,NaN,NaN,99,THANE,99999,NaN,INDIA,IN,...,187,44,204,150,73,271,164,170,246,2024-09-27
1,6e8e3227297409f3f33578400302825263cadc2ed0d1a0...,35-39,NaN,NaN,ZZ,Pune,ZZ,NaN,INDIA,411,...,187,44,204,150,73,271,164,170,246,2024-09-27
2,1b42f270aba756b64d7ae4e2409313097b0c91f7c2f2c7...,20-24,NaN,NaN,99,BHIWANI,95013,NaN,INDIA,IN,...,187,44,204,150,73,271,164,170,246,2024-09-27
3,06a4aae9b531a518260c7d0d88811cc202fd0d3e46d9ea...,20-24,NaN,NaN,NaN,CHIKMAGALUR,NaN,NaN,INDIA,NaN,...,187,44,204,150,73,271,164,170,246,2024-09-27
4,0e4fa92b7a41dc019c9f40457e180e94ca60d0b5c7128e...,30-34,NaN,NaN,NaN,NASIK,NaN,NaN,INDIA,NaN,...,187,44,204,150,73,271,164,170,246,2024-09-27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1285397,208d6130ed06bfa28102a553b800db738f348aee78ceed...,NaN,NaN,NaN,NaN,Chennai,NaN,NaN,INDIA,IN,...,68,291,85,31,318,152,45,51,127,2024-05-31
1285398,7ef63f0b16fa55ef0f2df11483014c166462c0433a2228...,45-49,NaN,NaN,ZZ,Mumbai,ZZ,NaN,INDIA,IN,...,124,347,141,87,10,208,101,107,183,2024-07-26
1285399,36de75588001e2282d38aee12b87919c7d0a359af49b5d...,30-34,NaN,NaN,NaN,DELHI,NaN,NaN,INDIA,NaN,...,68,291,85,31,318,152,45,51,127,2024-05-31
1285400,0beadfd5b2e548d6e4c5aaa730b7766b21ad5e5e0d88c3...,30-34,NaN,NaN,99,SURAT,95013,NaN,INDIA,IN,...,124,347,141,87,10,208,101,107,183,2024-07-26


In [ ]:
cdna_cleaned

,v24,v32,v44,v80,v82,v86,v87,v88,v92,v95,...,v45,v49,v50,v51,v52,v73,v229,v230,customer_code,batch_date
0,0.0,0.060637,1.000000,0.405941,1,0,0,1,0,0,...,True,False,True,True,True,False,43802.0,22101.0,635c3a7b6c89528607944fa312096e48c9f54dfaf821f5...,2024-11-29
1,0.0,0.039054,1.000000,0.415842,0,0,0,0,0,1,...,True,False,True,True,True,False,22101.0,22101.0,970ef77e5a753f82ed9ee1e4446db1dd29d237ed0f64c7...,2024-11-29
2,0.0,0.085303,1.000000,0.376238,0,0,0,0,0,0,...,True,False,True,True,True,False,43552.0,22101.0,ccc36bb7c74601863c7328faa90ade2d806bccea2a3c51...,2024-11-29
3,0.0,0.050360,0.666667,0.336634,1,0,0,1,0,1,...,True,False,True,False,False,False,22101.0,22101.0,3c34d39288675fb3033d5f24bfccf427f1ee97e8c4bbb3...,2024-11-29
4,0.0,0.039054,1.000000,0.287129,0,0,0,0,0,0,...,True,False,True,True,True,False,43552.0,43554.0,cb353c7e0419b7b9194979ba250e0dfd1fa19a41b01cb1...,2024-11-29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68445,0.0,0.026721,1.000000,0.396040,0,0,0,0,0,0,...,True,False,True,True,True,False,22101.0,22101.0,4cde59d5b42a2d221037cb2fca0de4b017b92486881c72...,2024-11-29
68446,0.0,0.039054,1.000000,0.435644,0,0,0,0,0,0,...,True,False,True,True,True,False,43513.0,43301.0,537705bf6fab6b290248264b5b34cc3f59352740d98964...,2024-11-29
68447,0.0,0.019527,1.000000,0.237624,0,0,0,0,0,0,...,True,False,True,True,False,False,22101.0,22101.0,48d22d3e52065cb6d54e1b88ec4d5a41b99f19be0427a2...,2024-11-29
68448,0.0,0.039054,1.000000,0.485149,0,0,0,0,0,0,...,False,False,True,True,True,False,43301.0,43301.0,5d8e50df07ecaeb2da40b588aab824e9d33a705e005a2b...,2024-11-29


In [ ]:
# Identify columns in CDNA with only one unique value
single_value_columns = [col for col in cdna_cleaned.columns if cdna_cleaned[col].nunique() == 1]

# Drop these columns from the dataset
cdna_cleaned2= cdna_cleaned.drop(columns=single_value_columns)

# Display the columns that were dropped
print(f"Columns dropped (unique value = 1): {single_value_columns}")

# Save the cleaned dataset
# cdna_cleaned.to_csv('cdna_cleaned.csv', index=False)

# Display information about the cleaned dataset
print(cdna_cleaned2.info())


Columns dropped (unique value = 1): []
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1285402 entries, 0 to 1285401
Data columns (total 40 columns):
 #   Column         Non-Null Count    Dtype   
---  ------         --------------    -----   
 0   v24            1285402 non-null  float64 
 1   v32            1285402 non-null  float64 
 2   v44            1285402 non-null  float64 
 3   v80            1285402 non-null  float64 
 4   v82            1285402 non-null  int64   
 5   v86            1285402 non-null  int64   
 6   v87            1285402 non-null  int64   
 7   v88            1285402 non-null  int64   
 8   v89            1285402 non-null  int64   
 9   v92            1285402 non-null  int64   
 10  v95            1285402 non-null  int64   
 11  v96            1285402 non-null  int64   
 12  v100           1285402 non-null  float64 
 13  v104           1285402 non-null  int64   
 14  v105           1285402 non-null  int64   
 15  v106           1285402 non-null  int64   
 1

In [ ]:
from sklearn.preprocessing import MinMaxScaler
import pandas as pd

# Assuming cdna_cleaned2 is already loaded
# Identify Numerical Columns
numerical_cols = cdna_cleaned2.select_dtypes(include=['float64', 'int64']).columns

# Categorize Columns
strict_binary_cols = [col for col in numerical_cols if sorted(cdna_cleaned2[col].dropna().unique()) == [0, 1]]
two_value_non_binary_cols = [
    col for col in numerical_cols
    if len(cdna_cleaned2[col].dropna().unique()) == 2 and col not in strict_binary_cols
]
other_numerical_cols = [col for col in numerical_cols if col not in strict_binary_cols + two_value_non_binary_cols]

print(f"Strict Binary Columns: {strict_binary_cols}")
print(f"Two-Value Non-Binary Columns: {two_value_non_binary_cols}")
print(f"Other Numerical Columns: {other_numerical_cols}")

# Scale Non-Binary Columns
scaler = MinMaxScaler()

# Scale two-value non-binary columns
if two_value_non_binary_cols:
    cdna_cleaned2[two_value_non_binary_cols] = scaler.fit_transform(cdna_cleaned2[two_value_non_binary_cols])

# Scale other numerical columns
if other_numerical_cols:
    cdna_cleaned2[other_numerical_cols] = scaler.fit_transform(cdna_cleaned2[other_numerical_cols])

# Verify Scaling
print("\nScaled Two-Value Non-Binary Columns:")
# print(cdna_cleaned2[two_value_non_binary_cols].describe())
print("\nScaled Other Numerical Columns:")
print(cdna_cleaned2[other_numerical_cols].describe())

# Final Dataset Info
print("\nFinal Dataset Info:")
print(cdna_cleaned2.info())

# Save Processed Dataset
# cdna_cleaned2.to_csv('processed_cdna_cleaned2_scaled.csv', index=False)


Strict Binary Columns: ['v24', 'v82', 'v86', 'v87', 'v88', 'v89', 'v92', 'v95', 'v96', 'v104', 'v105', 'v106']
Two-Value Non-Binary Columns: []
Other Numerical Columns: ['v32', 'v44', 'v80', 'v100', 'v224', 'v225', 'v227', 'v259', 'v263', 'v287', 'v288', 'v290', 'v291', 'v295', 'v298']

Scaled Two-Value Non-Binary Columns:

Scaled Other Numerical Columns:
                v32           v44           v80          v100          v224  \
count  1.285402e+06  1.285402e+06  1.285402e+06  1.285402e+06  1.285402e+06   
mean   5.281097e-02  9.008850e-01  2.931477e-01  1.987653e-02  5.498689e-02   
std    5.831806e-02  2.065985e-01  8.807706e-02  1.053138e-01  3.933557e-02   
min    0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00   
25%    3.374233e-02  1.000000e+00  2.320000e-01  0.000000e+00  3.267785e-02   
50%    5.010225e-02  1.000000e+00  2.720000e-01  0.000000e+00  4.822152e-02   
75%    5.419223e-02  1.000000e+00  3.360000e-01  0.000000e+00  6.528340e-02   
max    1.0

In [ ]:
df10['batch_date']=cdna_cleaned2

In [ ]:
df10.isnull().sum()

,0
v24,0
v32,0
v44,0
v80,0
v82,0
v86,0
v87,0
v88,0
v89,0
v92,0


In [ ]:
import pandas as pd
from tqdm import tqdm

def aggregate_rows_with_progress(df, group_col):
    # Initialize an empty DataFrame to store results
    aggregated_df = pd.DataFrame()

    # Group the DataFrame by the specified column
    grouped = df.groupby(group_col)

    # Iterate over groups with tqdm progress bar
    for customer_code, group in tqdm(grouped, desc="Aggregating rows"):
        # Aggregate numerical columns with mean
        numeric_data = group.select_dtypes(include=['float64', 'int64']).mean()

        # Aggregate categorical columns with mode
        categorical_data = {
            col: group[col].mode().iloc[0] if not group[col].mode().empty else None
            for col in group.select_dtypes(include=['object', 'category']).columns
        }

        # Aggregate datetime columns with median
        datetime_data = group.select_dtypes(include=['datetime64']).median()

        # Combine results into a single row
        aggregated_row = pd.concat([numeric_data, pd.Series(categorical_data), datetime_data])

        # Add the group key (customer_code)
        aggregated_row[group_col] = customer_code

        # Append to the final DataFrame
        aggregated_df = pd.concat([aggregated_df, pd.DataFrame([aggregated_row])], ignore_index=True)

    return aggregated_df

# Apply the aggregation function with progress bar
aggregated_df10 = aggregate_rows_with_progress(df10, group_col='customer_code')

# Display the aggregated DataFrame
print(aggregated_df10.head())

# Save the aggregated DataFrame if needed
# aggregated_df10.to_csv('aggregated_df10_with_progress.csv', index=False)


Streaming output truncated to the last 5000 lines.
<ipython-input-136-8a43492e42db>:32: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  aggregated_df = pd.concat([aggregated_df, pd.DataFrame([aggregated_row])], ignore_index=True)
Aggregating rows:  52%|█████▏    | 114603/220699 [20:57<30:32, 57.89it/s]<ipython-input-136-8a43492e42db>:32: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  aggregated_df = pd.concat([aggregated_df, pd.DataFrame([aggregated_row])], ignore_index=True)
Aggregating rows:  52%

In [ ]:
communication_history=pd.read_csv('/content/drive/MyDrive/data/train_action_history.csv')

In [ ]:
communication_history.drop(columns=['Offer_id','Offer_subid'	,'batch_id'	],inplace=True)

In [ ]:
# df20.isnull().sum()

,0
customer_code,0
send_timestamp,0
open_timestamp,733469
slot,0
category_ASSETS,0
...,...
sub_category_UPI,0
sub_category_VISTARA_ACQ,0
sub_category_VISTARA_UPGRADE,0
sub_category_VKYC COMPLETION,0


In [ ]:
# df20

,customer_code,product_category,product_sub_category,send_timestamp,open_timestamp
0,00be47975468d06b1e483afb3bd1271c7c17adb9841268...,CC_UPGRADE,UPI,2024-09-01T11:01:45.000Z,NaN
1,0177ba64f8025697f914c5f95004b11f910f39b7800317...,CC_ACTIVATION,30D_ACTIVATION,2024-11-02T14:48:31.000Z,2024-11-02T14:48:46.000Z
2,01b5cfa431cb5a553f17ba798d548125fd2f5c5210904c...,CC_SPENDS,PORTFOLIO,2024-10-26T16:56:19.000Z,2024-10-26T16:56:31.000Z
3,02066bb8e12006529edece807b88b4bdbb1c25b3fdee07...,CC_INORGANIC,CC_LOAN_ON_CARDS,2024-10-30T18:49:14.000Z,NaN
4,021ab34a32937f995eeba74fec48a8c6fd3e8c6c01de90...,CC_SPENDS,PORTFOLIO,2024-10-19T09:52:31.000Z,NaN
...,...,...,...,...,...
1456498,fd7ff5f48f0674b6f4d6a7dee6863d1e09d3d308f1974d...,CORPSAL,CORP SAL BALANCE BUILD UP,2024-10-05T10:00:23.000Z,NaN
1456499,fe2a1f0ece307ea99a78c61039e95904dc8ec2005dcb1d...,PL X-SELL,ACEPL,2024-10-30T17:45:25.000Z,2024-10-31T09:48:23.000Z
1456500,fe9ad21a47bd46f5857c6c8e03180b72299812752fcfd6...,CC_ACTIVATION,30D_ACTIVATION,2024-11-06T18:58:19.000Z,2024-11-06T22:19:25.000Z
1456501,ff15182b366a6b830f09f89dad54dfbeab5d047e76a726...,CC_SPENDS,PORTFOLIO,2024-10-22T09:45:27.000Z,NaN


In [ ]:
df20=communication_history
# df10=cdna_cleaned2

In [ ]:
# cdna_cleaned2

,v24,v32,v44,v80,v82,v86,v87,v88,v92,v95,...,v30,v45,v49,v50,v51,v52,v73,v229,v230,customer_code
0,0.0,0.060637,1.000000,0.405941,1,0,0,1,0,0,...,2.0,True,False,True,True,True,False,43802.0,22101.0,635c3a7b6c89528607944fa312096e48c9f54dfaf821f5...
1,0.0,0.039054,1.000000,0.415842,0,0,0,0,0,1,...,11.0,True,False,True,True,True,False,22101.0,22101.0,970ef77e5a753f82ed9ee1e4446db1dd29d237ed0f64c7...
2,0.0,0.085303,1.000000,0.376238,0,0,0,0,0,0,...,1.0,True,False,True,True,True,False,43552.0,22101.0,ccc36bb7c74601863c7328faa90ade2d806bccea2a3c51...
3,0.0,0.050360,0.666667,0.336634,1,0,0,1,0,1,...,2.0,True,False,True,False,False,False,22101.0,22101.0,3c34d39288675fb3033d5f24bfccf427f1ee97e8c4bbb3...
4,0.0,0.039054,1.000000,0.287129,0,0,0,0,0,0,...,95.0,True,False,True,True,True,False,43552.0,43554.0,cb353c7e0419b7b9194979ba250e0dfd1fa19a41b01cb1...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68445,0.0,0.026721,1.000000,0.396040,0,0,0,0,0,0,...,11.0,True,False,True,True,True,False,22101.0,22101.0,4cde59d5b42a2d221037cb2fca0de4b017b92486881c72...
68446,0.0,0.039054,1.000000,0.435644,0,0,0,0,0,0,...,11.0,True,False,True,True,True,False,43513.0,43301.0,537705bf6fab6b290248264b5b34cc3f59352740d98964...
68447,0.0,0.019527,1.000000,0.237624,0,0,0,0,0,0,...,2.0,True,False,True,True,False,False,22101.0,22101.0,48d22d3e52065cb6d54e1b88ec4d5a41b99f19be0427a2...
68448,0.0,0.039054,1.000000,0.485149,0,0,0,0,0,0,...,95.0,False,False,True,True,True,False,43301.0,43301.0,5d8e50df07ecaeb2da40b588aab824e9d33a705e005a2b...


In [ ]:
# import pandas as pd
# from tqdm import tqdm

# def aggregate_rows_with_progress(df, group_col):
#     # Initialize an empty DataFrame to store results
#     aggregated_df = pd.DataFrame()

#     # Group the DataFrame by the specified column
#     grouped = df.groupby(group_col)

#     # Iterate over groups with tqdm progress bar
#     for customer_code, group in tqdm(grouped, desc="Aggregating rows"):
#         # Aggregate numerical columns with mean
#         numeric_data = group.select_dtypes(include=['float64', 'int64']).mean()

#         # Aggregate categorical columns with mode
#         categorical_data = {
#             col: group[col].mode().iloc[0] if not group[col].mode().empty else None
#             for col in group.select_dtypes(include=['object', 'category']).columns
#         }

#         # Aggregate datetime columns with median
#         datetime_data = group.select_dtypes(include=['datetime64']).median()

#         # Combine results into a single row
#         aggregated_row = pd.concat([numeric_data, pd.Series(categorical_data), datetime_data])

#         # Add the group key (customer_code)
#         aggregated_row[group_col] = customer_code

#         # Append to the final DataFrame
#         aggregated_df = pd.concat([aggregated_df, pd.DataFrame([aggregated_row])], ignore_index=True)

#     return aggregated_df

# # Apply the aggregation function with progress bar
# aggregated_df10 = aggregate_rows_with_progress(df10, group_col='customer_code')

# # Display the aggregated DataFrame
# print(aggregated_df10.head())

# # Save the aggregated DataFrame if needed
# # aggregated_df10.to_csv('aggregated_df10_with_progress.csv', index=False)


                                       customer_code  \
0  00be47975468d06b1e483afb3bd1271c7c17adb9841268...   
1  0177ba64f8025697f914c5f95004b11f910f39b7800317...   
2  01b5cfa431cb5a553f17ba798d548125fd2f5c5210904c...   
3  02066bb8e12006529edece807b88b4bdbb1c25b3fdee07...   
4  021ab34a32937f995eeba74fec48a8c6fd3e8c6c01de90...   

             send_timestamp            open_timestamp  slot  category_ASSETS  \
0 2024-09-01 11:01:45+00:00                       NaN  25.0            False   
1 2024-11-02 14:48:31+00:00  2024-11-02T14:48:46.000Z  22.0            False   
2 2024-10-26 16:56:19+00:00  2024-10-26T16:56:31.000Z  23.0            False   
3 2024-10-30 18:49:14+00:00                       NaN  12.0            False   
4 2024-10-19 09:52:31+00:00                       NaN  21.0            False   

   category_CC_ACQ_SECURED  category_CC_ACQ_UNSECURED  category_CC_ACTIVATION  \
0                    False                      False                   False   
1                   

In [ ]:
# Ensure open_timestamp is in datetime format
# df20['send_timestamp'] = pd.to_datetime(df20['send_timestamp'], errors='coerce')

# Create a new column for the date-only part (YY-MM-DD) from open_timestamp
df20['send_date'] = df20['send_timestamp'].dt.date

# Display the updated DataFrame
# print(df20[['open_timestamp', 'open_date']].head())

# Save the updated DataFrame if needed
# df20.to_csv('df20_with_open_date.csv', index=False)


KeyError: 'open_timestamp'

In [ ]:
cdna_cleaned

,v24,v32,v44,v80,v82,v86,v87,v88,v92,v95,...,v45,v49,v50,v51,v52,v73,v229,v230,customer_code,batch_date
0,0.0,0.060637,1.000000,0.405941,1,0,0,1,0,0,...,True,False,True,True,True,False,43802.0,22101.0,635c3a7b6c89528607944fa312096e48c9f54dfaf821f5...,2024-11-29
1,0.0,0.039054,1.000000,0.415842,0,0,0,0,0,1,...,True,False,True,True,True,False,22101.0,22101.0,970ef77e5a753f82ed9ee1e4446db1dd29d237ed0f64c7...,2024-11-29
2,0.0,0.085303,1.000000,0.376238,0,0,0,0,0,0,...,True,False,True,True,True,False,43552.0,22101.0,ccc36bb7c74601863c7328faa90ade2d806bccea2a3c51...,2024-11-29
3,0.0,0.050360,0.666667,0.336634,1,0,0,1,0,1,...,True,False,True,False,False,False,22101.0,22101.0,3c34d39288675fb3033d5f24bfccf427f1ee97e8c4bbb3...,2024-11-29
4,0.0,0.039054,1.000000,0.287129,0,0,0,0,0,0,...,True,False,True,True,True,False,43552.0,43554.0,cb353c7e0419b7b9194979ba250e0dfd1fa19a41b01cb1...,2024-11-29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68445,0.0,0.026721,1.000000,0.396040,0,0,0,0,0,0,...,True,False,True,True,True,False,22101.0,22101.0,4cde59d5b42a2d221037cb2fca0de4b017b92486881c72...,2024-11-29
68446,0.0,0.039054,1.000000,0.435644,0,0,0,0,0,0,...,True,False,True,True,True,False,43513.0,43301.0,537705bf6fab6b290248264b5b34cc3f59352740d98964...,2024-11-29
68447,0.0,0.019527,1.000000,0.237624,0,0,0,0,0,0,...,True,False,True,True,False,False,22101.0,22101.0,48d22d3e52065cb6d54e1b88ec4d5a41b99f19be0427a2...,2024-11-29
68448,0.0,0.039054,1.000000,0.485149,0,0,0,0,0,0,...,False,False,True,True,True,False,43301.0,43301.0,5d8e50df07ecaeb2da40b588aab824e9d33a705e005a2b...,2024-11-29


In [ ]:
df10['batch_date']=cdna_cleaned['batch_date']

In [ ]:
# df10['batch_date'] = pd.to_datetime(df10['batch_date']).dt.tz_localize(None)
df20['send_timestamp'] = pd.to_datetime(df20['send_timestamp']).dt.tz_localize(None)

In [ ]:
df20.isnull().sum()

,0
customer_code,0
product_category,0
product_sub_category,0
send_timestamp,0
open_timestamp,765833
send_date,0


In [ ]:
df20['open_timestamp'] = pd.to_datetime(df20['open_timestamp'], errors='coerce')

# Create a new column for the date-only part (YY-MM-DD) from open_timestamp
df20['open_date'] = df20['open_timestamp'].dt.date

In [ ]:
df20

,customer_code,send_timestamp,open_timestamp,slot,category_ASSETS,category_CC_ACQ_SECURED,category_CC_ACQ_UNSECURED,category_CC_ACTIVATION,category_CC_INORGANIC,category_CC_REMARKETING,...,sub_category_SWYP_ACQ,sub_category_SWYP_UPGRADE,sub_category_TURBO-SA XSELL,sub_category_UNSECURED_ACQ,sub_category_UPI,sub_category_VISTARA_ACQ,sub_category_VISTARA_UPGRADE,sub_category_VKYC COMPLETION,send_date,open_date
0,00be47975468d06b1e483afb3bd1271c7c17adb9841268...,2024-09-01 11:01:45+00:00,NaT,25.0,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,2024-09-01,NaT
1,0177ba64f8025697f914c5f95004b11f910f39b7800317...,2024-11-02 14:48:31+00:00,2024-11-02 14:48:46+00:00,22.0,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,2024-11-02,2024-11-02
2,01b5cfa431cb5a553f17ba798d548125fd2f5c5210904c...,2024-10-26 16:56:19+00:00,2024-10-26 16:56:31+00:00,23.0,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,2024-10-26,2024-10-26
3,02066bb8e12006529edece807b88b4bdbb1c25b3fdee07...,2024-10-30 18:49:14+00:00,NaT,12.0,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,2024-10-30,NaT
4,021ab34a32937f995eeba74fec48a8c6fd3e8c6c01de90...,2024-10-19 09:52:31+00:00,NaT,21.0,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,2024-10-19,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1400055,fd7ff5f48f0674b6f4d6a7dee6863d1e09d3d308f1974d...,2024-10-05 10:00:23+00:00,NaT,21.0,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,2024-10-05,NaT
1400056,fe2a1f0ece307ea99a78c61039e95904dc8ec2005dcb1d...,2024-10-30 17:45:25+00:00,2024-10-31 09:48:23+00:00,11.0,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,2024-10-30,2024-10-31
1400057,fe9ad21a47bd46f5857c6c8e03180b72299812752fcfd6...,2024-11-06 18:58:19+00:00,2024-11-06 22:19:25+00:00,12.0,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,2024-11-06,2024-11-06
1400058,ff15182b366a6b830f09f89dad54dfbeab5d047e76a726...,2024-10-22 09:45:27+00:00,NaT,5.0,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,2024-10-22,NaT


In [ ]:
len(df20['customer_code'].unique())

64700

In [ ]:
df20['send_date'] = df20['send_date'].astype('datetime64[ns]')

In [ ]:
df20.isnull().sum()

,0
customer_code,0
product_category,0
product_sub_category,0
send_timestamp,0
open_timestamp,765833
send_date,0
open_date,765833


In [ ]:
df10['batch_date']

,batch_date
0,2024-11-29
1,2024-11-29
2,2024-11-29
3,2024-11-29
4,2024-11-29
...,...
68445,2024-11-29
68446,2024-11-29
68447,2024-11-29
68448,2024-11-29


In [ ]:
df100=df10

In [ ]:
# Remove rows with NaN in 'customer_code' before merging
df100_clean = df100.dropna(subset=['customer_code'])
df20_clean = df20.dropna(subset=['customer_code'])

# Initialize the progress bar
with tqdm(total=len(df100_clean) + len(df20_clean), desc="Merging datasets", unit="rows") as pbar:
    # Perform an outer join based on customer_code
    merged_df = pd.merge(
        df100_clean,  # Cleaned first dataset
        df20_clean,   # Cleaned second dataset
        on='customer_code',  # Common key for merging
        how='outer'  # Include all rows from both datasets
    )
    # Update the progress bar after merging
    pbar.update(len(df100_clean) + len(df20_clean))

# Display info about the merged DataFrame
print("\nMerged DataFrame Info:")
print(merged_df.info())

# Save the merged DataFrame to a CSV file
# merged_df.to_csv('merged_df100_df20_full_outer.csv', index=False)

# Display the first few rows of the merged DataFrame
print("\nFirst Few Rows of the Merged DataFrame:")
print(merged_df.head())


Merging datasets: 100%|██████████| 1524953/1524953 [00:01<00:00, 1284198.54rows/s]



Merged DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460253 entries, 0 to 1460252
Data columns (total 39 columns):
 #   Column                Non-Null Count    Dtype              
---  ------                --------------    -----              
 0   v24                   1460253 non-null  float64            
 1   v32                   1460253 non-null  float64            
 2   v44                   1460253 non-null  float64            
 3   v80                   1460253 non-null  float64            
 4   v82                   1460253 non-null  int64              
 5   v86                   1460253 non-null  int64              
 6   v87                   1460253 non-null  int64              
 7   v88                   1460253 non-null  int64              
 8   v92                   1460253 non-null  int64              
 9   v95                   1460253 non-null  int64              
 10  v96                   1460253 non-null  int64              
 11  v100         

In [ ]:
merged_df.isnull().sum()

,0
v24,0
v32,0
v44,0
v80,0
v82,0
v86,0
v87,0
v88,0
v92,0
v95,0


In [ ]:
# from tqdm import tqdm
# import pandas as pd

# # Initialize the progress bar
# with tqdm(total=len(df100) + len(df20), desc="Merging datasets", unit="rows") as pbar:
#     # Perform an outer join based on customer_code
#     merged_df = pd.merge(
#         df100,  # First dataset
#         df20,  # Second dataset
#         on='customer_code',  # Common key for merging
#         how='outer'  # Include all rows from both datasets
#     )
#     # Update the progress bar after merging
#     pbar.update(len(df100) + len(df20))

# # Display info about the merged DataFrame
# print("\nMerged DataFrame Info:")
# print(merged_df.info())

# # Save the merged DataFrame to a CSV file
# merged_df.to_csv('merged_df100_df20_full_outer.csv', index=False)

# # Display the first few rows of the merged DataFrame
# print("\nFirst Few Rows of the Merged DataFrame:")
# print(merged_df.head())


Merging datasets: 100%|██████████| 1524953/1524953 [00:01<00:00, 962781.91rows/s]



Merged DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460253 entries, 0 to 1460252
Data columns (total 39 columns):
 #   Column                Non-Null Count    Dtype              
---  ------                --------------    -----              
 0   v24                   1460253 non-null  float64            
 1   v32                   1460253 non-null  float64            
 2   v44                   1460253 non-null  float64            
 3   v80                   1460253 non-null  float64            
 4   v82                   1460253 non-null  int64              
 5   v86                   1460253 non-null  int64              
 6   v87                   1460253 non-null  int64              
 7   v88                   1460253 non-null  int64              
 8   v92                   1460253 non-null  int64              
 9   v95                   1460253 non-null  int64              
 10  v96                   1460253 non-null  int64              
 11  v100         

In [ ]:
len(merged_df['customer_code'].unique())

68450

In [ ]:
merged_df.isnull().sum()

,0
v24,0
v32,0
v44,0
v80,0
v82,0
v86,0
v87,0
v88,0
v92,0
v95,0


In [ ]:
import pandas as pd

# Ensure timestamps are properly converted to datetime format
merged_df['send_timestamp'] = pd.to_datetime(merged_df['send_timestamp'], errors='coerce')
merged_df['open_timestamp'] = pd.to_datetime(merged_df['open_timestamp'], errors='coerce')

# Function to calculate 28 slots based on timestamp
def calculate_28_slots(timestamp):
    if pd.isnull(timestamp):
        return None  # Not sent/opened
    day_of_week = timestamp.weekday()  # Monday=0, Sunday=6
    hour = timestamp.hour

    # Map hours to slots
    if 9 <= hour < 12:
        time_slot = 1
    elif 12 <= hour < 15:
        time_slot = 2
    elif 15 <= hour < 18:
        time_slot = 3
    elif 18 <= hour < 21:
        time_slot = 4
    else:
        return None  # Outside defined slots

    # Calculate the overall slot number
    return day_of_week * 4 + time_slot  # Combine day (0-6) and slot (1-4)

# Apply the slot calculation function
merged_df['send_slot'] = merged_df['send_timestamp'].apply(calculate_28_slots)
merged_df['open_slot'] = merged_df['open_timestamp'].apply(calculate_28_slots)

# Compare `send_slot` and `open_slot` and create a flag
merged_df['flag'] = merged_df.apply(
    lambda row: 1 if row['send_slot'] == row['open_slot'] else 0, axis=1
)

# Display the updated DataFrame
print("\nUpdated DataFrame with Send Slot, Open Slot, and Flag:")
print(merged_df[['customer_code', 'send_slot', 'open_slot', 'flag']].head())

# # Save the updated DataFrame
# merged_df.to_csv('updated_merged_data_with_slots_and_flag.csv', index=False)
# print("\nUpdated DataFrame saved as 'updated_merged_data_with_slots_and_flag.csv'.")



Updated DataFrame with Send Slot, Open Slot, and Flag:
                                       customer_code  send_slot  open_slot  \
0  0001669127bc294ff6ed183ac433afab0fa11d4693a844...        2.0        NaN   
1  0001669127bc294ff6ed183ac433afab0fa11d4693a844...       11.0        NaN   
2  0001669127bc294ff6ed183ac433afab0fa11d4693a844...       22.0        NaN   
3  0001669127bc294ff6ed183ac433afab0fa11d4693a844...        3.0        NaN   
4  0001669127bc294ff6ed183ac433afab0fa11d4693a844...        9.0        NaN   

   flag  
0     0  
1     0  
2     0  
3     0  
4     0  


In [ ]:
merged_df.isnull().sum()

,0
v24,0
v32,0
v44,0
v80,0
v82,0
v86,0
v87,0
v88,0
v92,0
v95,0


In [ ]:
# Drop only columns that exist in the DataFrame
columns_to_drop = ['batch_date', 'open_date', 'send_date']
merged_df = merged_df.drop(columns=[col for col in columns_to_drop if col in merged_df.columns])
# merged_df.to_csv('updated_merged_data_with_slots_and_flag.csv', index=False)



In [ ]:
merged_df.to_csv('/content/drive/MyDrive/data/TESTFINALDATA.csv', index=False)
